In [ ]:
import cudf
import cupy as cp

import pandas as pd
import numpy as np

user - время, затраченное на сами вычисления

sys - время, затраченное всеми процессорами на выполнение связанных с системой задач, таких как выделение памяти, например.

О датасете:
    
База данных системы эпиднадзора за случаями заболевания COVID-19 включает данные, сообщаемые штатами США и автономными отчетными органами, включая Нью-Йорк и округ Колумбия (округ Колумбия).

https://www.kaggle.com/arashnic/covid19-case-surveillance-public-use-dataset

current_status - текущий статус человека, sex - пол, age_group - возростная группа, Race and ethnicity (combined) - демографическая группа, hosp_yn - был ли пациент госпитализирован, icu_yn - был ли пациент госпитализирован в отделение интенсивной терапии,
death_yn - умер ли пациент от болезни, medcond_yn - были ли какие-то сопутствующие заболевания или риск-факторы?

In [ ]:
%time df = pd.read_parquet('covid.gzip')

In [ ]:
df

In [ ]:
%time df.isna().sum()

In [ ]:
%time df = df.dropna()

Из-за сложного управления памятью GPU в cuDF первая загрузка данных в новую среду памяти RAPIDS иногда значительно медленнее, чем последующие загрузки. Диспетчер памяти RAPIDS готовит дополнительную память для размещения массивов, которые вы можете использовать с данными, вместо того чтобы многократно выделять и освобождать память на протяжении всего рабочего процесса.
Другими словами, как и в большинсвте случаев, память надо "прогреть".

In [ ]:
%time gdf = cudf.read_parquet('covid.gzip')

In [ ]:
%time gdf.isna().sum()

In [ ]:
%time gdf = gdf.dropna()

In [ ]:
gdf

Давайте измерим время фильтрации датасета. Большая часть времени тратится на выполнение системных операций, таких как выделение памяти, например. А вот само время, затраченное на вычисления очень мало.

In [ ]:
%time male_df = df.loc[df['sex']=='Male']

In [ ]:
%time male_gdf = gdf.loc[gdf['sex']=='Male']

А теперь давайте посмотрим на то, как распределены данные по возрастам

In [ ]:
%time df['age_group'].value_counts(normalize=True)

In [ ]:
%time gdf['age_group'].value_counts(normalize=True)

А как сочетать в себе cupy и cudf?

In [ ]:
%time non_hisp_30_df = df.loc[np.logical_and(df['Race and ethnicity (combined)'].str.endswith('Non-Hispanic'),\
                                             df['age_group'].str.startswith('30'))]

In [ ]:
%time non_hisp_30_gdf = gdf.loc[cp.logical_and(gdf['Race and ethnicity (combined)'].str.endswith('Non-Hispanic'),\
                                               gdf['age_group'].str.startswith('30'))]

Группировка данных

In [ ]:
df

In [ ]:
df['death_yn'].value_counts()

Давайте создадим новую переменную, которая ставит индикатор выздоровления всегда, даже когда информации нет

Плохая новость - udf в cudf нельзя использовать для типов str и category

In [ ]:
df['death_ind'] = df['death_yn'].apply(lambda x: 1 if x=='Yes' else 0)

In [ ]:
gdf['death_ind'] = gdf['death_yn'].applymap(lambda x: 1 if x=='Yes' else 0)

In [ ]:
gdf['death_ind'] = (gdf['death_yn']=='Yes') * 1

Давайте посмотрим, есть ли различие в вероятности смерти, если есть осложения

In [ ]:
gdf['death_ind'].mean()

In [ ]:
%time df[['medcond_yn', 'death_ind']].groupby(['medcond_yn']).mean()

In [ ]:
%time gdf[['medcond_yn', 'death_ind']].groupby(['medcond_yn']).mean()

# Домашнее задание

Сравните среднюю вероятность смерти мужчин и женщин по группам возростов на основе столбца death_ind. Тоже самое проделайте для вероятности госпитализации, преобразовав переменную hosp_yn, как мы сделали с переменной death_yn. Используйте cudf и сохраните результат на диск.

## Кстати

Можно писать собственные udf через numba

In [ ]:
from cudf.datasets import randomdata

df = randomdata(nrows=10, dtypes={'a':float, 'b':float, 'c':str}, seed=12)
df.head()

In [ ]:
from numba import cuda

@cuda.jit
def multiply(in_col, out_col, multiplier):
    i = cuda.grid(1)
    if i < in_col.size:
        out_col[i] = in_col[i] * multiplier

In [ ]:
size = len(df['a'])
df['e'] = 0.0
multiply.forall(size)(df['a'], df['e'], 3.0)

In [ ]:
df

В сuDF есть хорший метод apply_rows, который позволяет писать udf на python и применять к нескольким столбцам

In [ ]:
def conditional_add(x, y, out):
    for i, (a, e) in enumerate(zip(x, y)):
        if a > 0:
            out[i] = a + e
        else:
            out[i] = a

In [ ]:
df = df.apply_rows(conditional_add,
                   incols={'a':'x', 'b':'y'},
                   outcols={'out': np.float64},
                   kwargs={}
                  )
df